# Glacier Gallery

## Import packages

In [ ]:
import panel as pn
import pandas as pd
import holoviews as hv
import geoviews as gv
from holoviews.plotting.links import Link
from holoviews.plotting.bokeh import LinkCallback
import geoviews.tile_sources as gts
from bokeh.models import HoverTool

hv.extension('bokeh', width=100)
pn.extension()
gv.extension('bokeh')

## Choose language for html document!

In [ ]:
language = 'en'  # 'en' or 'de'
file_name = f'gallery-app_{language}.html'

### Import text in different languages and selection of glacier with meta data

In [ ]:
# import all texts in the corresponding language from here:
from international import trads, supported_languages
from glacier_data import glaciers, links

### Function to prepare coordinates

In [ ]:
import math
from ast import literal_eval

def merc(Coords):
    lat = Coords[0]
    lon = Coords[1]

    r_major = 6378137.000
    x = r_major * math.radians(lon)
    scale = x/lon
    y = 180.0/math.pi * math.log(math.tan(math.pi/4.0 + 
        lat * (math.pi/180.0)/2.0)) * scale    
    return (x, y)

## Load data

In [ ]:
df_all = pd.read_hdf('./data/rgi62_era5_itmix_df.h5', 'df')
df_all['rgi_id']= list(df_all.axes[0])
df_sel = df_all[['rgi_id', 'CenLon', 'CenLat', 'Lmax', 'GlacierType',
                 'TerminusType', 'GlacierStatus', 'IsTidewater',
                 'IsNominal' ]].copy()

## Select glaciers, add photos and build lists with information about them

In [ ]:
Glacier_No = glaciers # imported glaciers and meta data from Glaciers.py

# Create lists from glacier list above
glacier_list = []
glacier_names = []
gltype = []
glacier_pics =[]
glpic_pretext_source = []
glpic_cite = []
glpic_lic = []
glpic_wi = []
glpic_he = []
glpic_src = []
for gl_no in Glacier_No:
    glacier_list.append(gl_no[0])
    glacier_names.append(gl_no[1])
    gltype.append(gl_no[2])
    glacier_pics.append(gl_no[3])
    glpic_pretext_source.append(gl_no[4])
    glpic_cite.append(gl_no[5])
    glpic_lic.append (gl_no[6])
    glpic_wi.append(gl_no[8])
    glpic_he.append(gl_no[9])
    glpic_src.append(gl_no[10])

# create a new dataframe with the selected glaciers from the loaded data about glaciers 
df = df_sel.loc[df_sel['rgi_id'].isin(glacier_list)].copy()
df['name'] = glacier_names
df['Coords'] = list(zip(round(df.CenLat,2), round(df.CenLon,2)))
df['type'] = gltype
df['img'] = glacier_pics
df['pic_pretext'] = glpic_pretext_source
df['glpic_lic'] = glpic_lic
df['pic_cite'] = glpic_cite
df['pic_width'] = glpic_wi
df['pic_height'] = glpic_he
df['pic_src'] = glpic_src

## Define texts in according language

In [ ]:
title                 = '<div style="font-size:38px; color: #326a86; font-weight: bold" >{}</div>'.format(trads['Title_hp'][language])
instruction           = '<div style="font-size:15px; color: #326a86" >{}</div>'.format(trads['instruction'][language])
link_to_license_intro = '<div>{}</div>'.format(trads['link_to_license_intro'][language])
links_pic_src_intro   = '<div>{}</div>'.format(trads['links_pic_src_intro'][language])

# tab titles
tab_title1 = trads['tab_title_1'][language]
tab_title2 = trads['tab_title_2'][language]

links_to_license = links['links_license']
links_pic_src    = links['links_pics']

df['Def'] = trads['Glacier_definition'][language]
df['Title'] = trads['Glacier_title'][language]
df['text_source'] = trads['Glacier_text_source'][language]

df['no_glacier_selected'] = trads['no_glacier_selected'][language]

# translate "terminating" entry:
df.loc[df['TerminusType'] == 'Land-terminating', 'TerminusType']   = trads['TerminusType_land'][language]
df.loc[df['TerminusType'] == 'Marine-terminating', 'TerminusType'] = trads['TerminusType_marine'][language]

# define colors for dots of different glacier types
glacier_type_colors = {trads['type_pure_ice'][language]: '#56B4E9',
                       trads['type_calving'][language]: '#F0E442',
                       trads['type_debris_covered'][language]: '#009E73',
                       trads['type_ice_cap'][language]: '#D55E00'}

### Add texts about glacier types to the dataframe

In [ ]:
df.loc[df['type'] == 'Pure ice glacier',       'glaciertype_text'] = trads['pure_ice_text'][language]
df.loc[df['type'] == 'Calving glacier',        'glaciertype_text'] = trads['calving_text'][language]
df.loc[df['type'] == 'Debris covered glacier', 'glaciertype_text'] = trads['debris_text'][language]
df.loc[df['type'] == 'Ice cap',                'glaciertype_text'] = trads['ice_cap_text'][language]
df.loc[df['type'] == 'Pure ice glacier',       'glaciertype_source'] = trads['pure_ice_source'][language]
df.loc[df['type'] == 'Calving glacier',        'glaciertype_source'] = trads['calving_glacier_source'][language]
df.loc[df['type'] == 'Debris covered glacier', 'glaciertype_source'] = trads['debris_glacier_source'][language]
df.loc[df['type'] == 'Ice cap',                'glaciertype_source'] = trads['ice_cap_source'][language]

# translate "type" entry:
df.loc[df['type'] == 'Pure ice glacier', 'type']       = trads['type_pure_ice'][language]
df.loc[df['type'] == 'Calving glacier', 'type']        = trads['type_calving'][language]
df.loc[df['type'] == 'Debris covered glacier', 'type'] = trads['type_debris_covered'][language]
df.loc[df['type'] == 'Ice cap', 'type']                = trads['type_ice_cap'][language]

# translate "pic_pretext":
df.loc[df['pic_pretext'] == 'Photo courtesy', 'pic_pretext'] = trads['pic_pretext'][language]

## Build special datastructure with all information

In [ ]:
# prepare coordinates  
df.loc[:, 'coords_x'] = df['Coords'].apply(lambda x: merc(x)[0])
df.loc[:, 'coords_y'] = df['Coords'].apply(lambda x: merc(x)[1])

df.loc[:, 'LonDeg'] = df['CenLon']
df.loc[:, 'LatDeg'] = df['CenLat']

## Design for map with hover and webpage

In [ ]:
# background map
background_map = gts.tile_sources['CartoLight']

# define hover with image and values to display
TOOLTIPS =  """
    <div>
        <div>
            <img
                src="@img" 
                height="@pic_height" 
                alt="image loading..." 
                width="@pic_width"
                style=" margin: 2px 2px 0px 2px;"
                border="2">
            </img>
        </div>
        <div>
                <span style="font-size: 17px; font-weight: bold;">@name                </span> <br>
                <span style="font-size: 15px;                   ">@type                </span> <br>
                <span style="font-size: 15px;                   ">{}:              </span>
                <span style="font-size: 15px;                   ">@Lmax                </span>
                <span style="font-size: 15px;                   ">m                    </span> <br>
                <span style="font-size: 15px;                   ">@TerminusType        </span> <br>
                <span style="font-size: 15px;                   ">{}:            </span>
                <span style="font-size: 13px; color: #696;      ">(@LatDeg, @LonDeg)   </span> <br>
                <span style="font-size: 9px;                    ">@pic_pretext @pic_cite @glpic_lic </span>
        </div>
    </div>
            """.format(trads['tooltip_length'][language], trads['tooltip_loc'][language])
hover = HoverTool(tooltips=TOOLTIPS)

# glacier points, include all needed data (everything from hover info to text description) in vdims
glacier_points = gv.Points(df,
                           kdims=['CenLon', 'CenLat'],
                           vdims=['type', 'name', 'Lmax', 'TerminusType', 'pic_pretext', 'pic_cite',
                                  'glpic_lic', 'img', 'pic_height', 'pic_width', 'LatDeg', 'LonDeg',
                                  'glaciertype_text', 'glaciertype_source', 'no_glacier_selected']
                          ).opts(default_tools=['tap', 'wheel_zoom', 'reset', 'pan', hover],
                                 size=10,
                                 line_color='black',
                                 fill_alpha=0.9,
                                 fill_color='type',
                                 cmap=glacier_type_colors,
                                 xaxis=None,
                                 yaxis=None
                                )

# put together the total figure
total_map = (glacier_points * background_map).opts(width=1000, height=650)

# text which is changed depending on selected point
type_text = hv.Div(df['no_glacier_selected'].values[0]
                  ).opts(sizing_mode='stretch_width',
                         height=10)

glaciertype_text = hv.Div(''
                         ).opts(sizing_mode='stretch_both')

glaciertype_source_text = hv.Div(''
                                ).opts(sizing_mode='stretch_both')


# defining the links between the plot and the text
# currently the text consists of three parts with three links
# because html formatting is not working, if this is fixed
# we can reduce it to one text object with one link
# (see https://github.com/holoviz/holoviews/issues/5590)
class TypeTextLink(Link):
    _requires_target = True

class TypeTextCallback(LinkCallback):

    source_model = 'selected'
    source_handles = ['cds']
    on_source_changes = ['indices']

    target_model = 'plot'
    
    source_code = """
        var inds = source_selected.indices

        if (inds.length == 0) {
            target_plot.text = source_cds.data['no_glacier_selected'][0]
        } else {
            target_plot.text = source_cds.data['type'][inds[0]]
        }
    """

TypeTextLink.register_callback('bokeh', TypeTextCallback)
TypeTextLink(glacier_points, type_text)

class GlaciertypeTextLink(Link):
    _requires_target = True

class GlaciertypeTextCallback(LinkCallback):

    source_model = 'selected'
    source_handles = ['cds']
    on_source_changes = ['indices']

    target_model = 'plot'
    
    source_code = """
        var inds = source_selected.indices

        if (inds.length == 0) {
            target_plot.text = ' '
        } else {
            target_plot.text = source_cds.data['glaciertype_text'][inds[0]]
        }
    """

GlaciertypeTextLink.register_callback('bokeh', GlaciertypeTextCallback)
GlaciertypeTextLink(glacier_points, glaciertype_text)

class GlaciertypeSourceTextLink(Link):
    _requires_target = True

class GlaciertypeSourceTextCallback(LinkCallback):

    source_model = 'selected'
    source_handles = ['cds']
    on_source_changes = ['indices']

    target_model = 'plot'
    
    source_code = """
        var inds = source_selected.indices

        if (inds.length == 0) {
            target_plot.text = ' '
        } else {
            target_plot.text = source_cds.data['glaciertype_source'][inds[0]]
        }
    """

GlaciertypeSourceTextLink.register_callback('bokeh', GlaciertypeSourceTextCallback)
GlaciertypeSourceTextLink(glacier_points, glaciertype_source_text)

# text which stays the same
constant_text = hv.Div(f'<h2>{glacier_points.data.Title.iloc[0]}</h2>'
                       f'{glacier_points.data.Def.iloc[0]}<br><br>'
                       f'<i>{glacier_points.data.text_source.iloc[0]}</i>'
                      ).opts(sizing_mode='stretch_both')

# put together tab with texts
tabs = pn.Tabs((tab_title1,
                pn.Column(type_text,
                          pn.Spacer(height=10),
                          glaciertype_text,
                          pn.Spacer(height=10),
                          glaciertype_source_text,
                          sizing_mode='stretch_both')
               ),
               (tab_title2, constant_text),
               width=330)

#### Logos

In [ ]:
oggm_logo   = '<a href="http://edu.oggm.org"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/oggm_loupe.png" width=220></a>'
hv_logo = '<p style="margin-top: 0px;margin-bottom: 0px;"><a href="https://holoviz.org"><img src="https://holoviz.org/assets/holoviz-logo-stacked.svg" width=80></a></p>'
fk_logo     = '<a href="https://www.uibk.ac.at/foerderkreis1669/"><img src="https://raw.githubusercontent.com/zschirmeister/glacier-gallery/master/uibk_logo_narrow.png" width=140></a>'

## Design application

In [ ]:
# Panel
header = pn.Row(pn.Pane(oggm_logo),  pn.layout.Spacer(width=30), 
                pn.Column(pn.Row(pn.Spacer(width=130),
                                 pn.Pane(title,
                                         width=400)),
                          pn.Row(pn.Spacer(width=80),
                                 pn.Pane(instruction,
                                         width=1200))),
                pn.layout.HSpacer(),
                sizing_mode='stretch_width'
                )
logos = pn.Column(pn.Spacer(height=30),
                  pn.Pane(hv_logo,
                          width=100),
                  pn.Spacer(height=40),
                  pn.Pane(fk_logo))
links = pn.Column(link_to_license_intro,
                  links_to_license,
                  links_pic_src_intro,
                  links_pic_src,
                  sizing_mode='stretch_width')

# Put plot and text together and show it
app = pn.Column(header,
                pn.Row(logos,
                       pn.Column(pn.Spacer(height=22),
                                 total_map),
                       pn.Spacer(width=2),
                       tabs,
                       sizing_mode='stretch_width'),
                pn.Spacer(height=60),
                links,
                sizing_mode='stretch_width')
#app.show()

In [ ]:
# save as a stand-alone HTML
app.save(file_name, 'Glacier Gallery')